# An introduction to statistical distributions

# Import modules
___

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sp

plt.style.use('custom.mplstyle')
COLORS = plt.rcParams['axes.prop_cycle'].by_key()['color']
%config InlineBackend.figure_format = 'retina'

# Ideas covered
___

We seek to understand probability distributions which characterize fundamental processes in statistics (and physics) and can be used as (baseline) models for real-world phenomena. We will look at the following distributions:

- [Binomial](https://en.wikipedia.org/wiki/Binomial_distribution)
- [Normal/Gaussian](https://en.wikipedia.org/wiki/Normal_distribution)

The hyperlinks will direct you to the Wikipedia pages for each distribution. These mathematical Wikipedia pages typically contain a wealth of information. They are useful for developing your intuition for these distributions and are references for neat properties.

# Random number generators
___

To study statistical distributions, we need a way of generating randomness. One way computers can generate randomness is by using a [pseudorandom number generator](https://en.wikipedia.org/wiki/Pseudorandom_number_generator). The nitty-gritty details of this topic aren't necessary to work through this notebook. However, a few things should be understood here:

- **Psuedorandom numbers aren't actually random!**
- Sequences of pseudorandom numbers are determined by a *seed*. This seed enables *reproducibility*.

We initialize a pseudorandom number generation (```rng```) with ```seed=0``` in the following way:

In [ ]:
rng = np.random.default_rng(seed=0)

Calling `rng.random()` produces a random number $\in [0, 1]$.

In [ ]:
rng.random()

To see how this random number generator is reproducible, generate three random numbers, recreate a `default_rng` object with the same seed, generate three random numbers again. Then generate three more random numbers to see the sequence continued. Change the seed and observe the same sequence of numbers is always produced when random number generators have the same seed.

In [ ]:
seed = 0

rng = np.random.default_rng(seed=seed)
print('Initialized first random number generator.')
for i in range(3):
    print(rng.random())
    
print('\nInitialized second random number generator with same seed.')
rng = np.random.default_rng(seed=seed)
for i in range(3):
    print(rng.random())
    
print('\nWithout reinitializing, rng will continue generating random numbers according to its sequence.')
for i in range(3):
    print(rng.random())

The point here is don't keep initializing your `rng`! Set it once, and use it!

### An important note on function parameters

When we initialized numpy arrays, we used a parameter called `shape` to specify the size of the dimensions. E.g.,

```python
np.zeros(shape=10)
np.ones(shape=(4, 50, 25))
```

When generating random numbers using `rng`, we can generate many random numbers at once. In fact, it is recommended and much more efficient to generate all the random numbers you will need rather than generating them one at a time. However, instead of using the `shape` parameter, we use the `size` parameter. (You can check various numpy docs.) This `size` parameter is used across all numpy functions for random number generation.

Try generating 10 random numbers using `shape` and then try using `size`.

In [ ]:
rng.random(shape=10)

In [ ]:
rng.random(size=10)

We can generate random numbers in a multidimensional way easily.

In [ ]:
rng.random(size=(3, 4, 2))

# The binomial distribution

A binomial random variable is a sum of Bernoulli random variables. Let $B$ be a Bernoulli random variable and $B'$ be a binomial random variable. Then for a binomial process with $n$ trials, we have

$$
B' = \sum_{i=1}^{n} B_i
$$

As a reminder, for a Binomial process with parameters $n$, the number of Bernoulli trials (coin flips), and $p$, the probability of a seeing a heads, the probability of observing $k \in \{0, 1, 2, 3, \ldots, n \}$ heads is

$$
P(k | p, n) = {n \choose k} p^k (1-p)^{n-k}
$$

## Numpy interlude

As stated above, we can simulate many random numbers at the same time. Let's exploit that functionality. Hopefully, the ensuing code will be more intutive.

Let's simulate $R$ binomial processes which each have $n$ Bernoulli trials (coin flips). We can do so by generating an $R \times n$ matrix of random numbers.

|                    | Bernoulli trial 1 | Bernoulli trial 2 | Bernoulli trial 3 | ... |  Bernoulli trial $n$ |
| -----------        | -----------       |    ----------     | -----------       | ----------- | ------|
| Binom process 1    | 0.252             | 0.842             | 0.928             | ...      | 0.125  |
| Binom process 2    | 0.484             | 0.238             | 0.101             | ...      | 0.589  |
| Binom process 3    | 0.661             | 0.747             | 0.747             | ...      | 0.324  |
| ...                | ...               | ...               | ...               | ...      | ...    |
| Binom process $R$  | 0.313             | 0.933             | 0.027             | ...      | 0.481  |

Then we can check which random numbers correspond to heads or tails. Random numbers which are successes are those with value $\leq p$. For simplicity, let's take $p=0.5$

|                    | Bernoulli trial 1 | Bernoulli trial 2 | Bernoulli trial 3 | ... |  Bernoulli trial $n$ |
| -----------        | -----------       |    ----------     | -----------       | ----------- | ------|
| Binom process 1    | H                 | T                 | T                 | ...      | H  |
| Binom process 2    | H                 | H                 | T                 | ...      | T  |
| Binom process 3    | T                 | T                 | T                 | ...      | H  |
| ...                | ...               | ...               | ...               | ...      | ...    |
| Binom process $R$  | H                 | T                 | H                 | ...      | H  |

Programmatically, we can implement this using bools (whether something is True or False). Let's rephrase our coin flip experiment from, "Heads or tails?" to, "Is the outcome a heads?" Then we can change our label from heads or tails to True (we observed a heads) or False (we observed a tails). T = True and F = False in the following table.

|                    | Bernoulli trial 1 | Bernoulli trial 2 | Bernoulli trial 3 | ... |  Bernoulli trial $n$ |
| -----------        | -----------       |    ----------     | -----------       | ----------- | ------|
| Binom process 1    | T                 | F                 | F                 | ...      | T  |
| Binom process 2    | T                 | T                 | F                 | ...      | F  |
| Binom process 3    | F                 | F                 | F                 | ...      | T  |
| ...                | ...               | ...               | ...               | ...      | ...    |
| Binom process $R$  | T                 | F                 | T                 | ...      | T  |

Next, we need to count how many heads we have for each binomial processes. We count each how many Trues are in each row and get our value. For simplicity, let's assume $n=4$ and take the values from above.

|                    | $k$ | 
| -----------        | -----------       | 
| Binom process 1    | 2                 |
| Binom process 2    | 2                 |
| Binom process 3    | 1                 | 
| ...                | ...               |
| Binom process $R$  | 3                 | 


### Write a function to simulate a binomial process.

It should have four parameters
- `rng`, the random number generator
- `p`, the probability of a success
- `num_trials`, the number of Bernoulli trials for each binomial process
- `num_realizations`, the amount of binomial processes/experiments to simulate

It should return
- `outcomes`, a numpy array containing how many successes were in each binomial process
- `distribution`, a numpy array containing the binomial probabilities

We use [`np.sum`](https://numpy.org/doc/stable/reference/generated/numpy.sum.html) which sums numpy arrays along whichever dimension we specify and [`np.bincount`](https://numpy.org/doc/stable/reference/generated/numpy.bincount.html) which counts the number of occurrences for each non-negative integer. We also use [`np.trim_zeros`](https://numpy.org/doc/stable/reference/generated/numpy.trim_zeros.html) to clean things up a bit.

Let's build up the function before we write it. Let's simulate 4 binomial processes (experiments) with $p=0.5$ and $n=6$, so a fair coin and 6 coin flips per experiment. We begin by making a matrix of random numbers.

We convert these probabilities into heads (True, we did see a head) and tails (False, we did not see a head).

True and False are booleans. This means that True = 1 and False = 0. (They are binary values.) We can exploit this behavior. Because a binomial process is the sum of how many heads we see, we sum all the columns for each row. We use `np.sum` and the keyword `axis=1` to tell it to sum over all the columns for each row.

Does this match how many Trues are in each row?

This gives us our Binomial random numbers. We could be done here. But let's also calculate the probabilities (frequenicies) of each number in our array. We'll use this later.

Simulating a Binomial process gives us an non-negative integer, how many heads we observed after some coin flips. We use `np.bincount` which counts the number of occurrences for each non-negative integer in an array. I.e., the number in the 0th position is how many 0s there were in the array, the number in the 1st position is how many 1s there were in the array, the number in the 2nd position is how many 2s there were in the array, etc.

We have the number of occurrences, but we'd like to convert these number of occurrences to probabilities/frequencies. We need to divide these numbers by how many experiments we performed in total. We could divide by $n$ or use `binom_rands.shape[0]` which also tells us how many experiments performed. (`binom_rands.shape[0]` gives how many numbers are in the array `binom_rands`.)

Finally, because of how we're going to write the functions below, let's get of the trailing 0s at either end of the array.

Now, let's construct a function using the above.

In [ ]:
def my_binomial(rng, p, num_trials, num_realizations):
    pass

Let's see what this returns for $p=0.3$, `num_trials` $= 5$, `num_realizations`$=4$.

In [ ]:
my_binomial(rng, 0.3, 5, 4)

Let's see what this returns for $p=0.3$, `num_trials` $= 100$, `num_realizations`$=4$.

In [ ]:
my_binomial(rng, 0.3, 100, 4)

### Run your function using $p=0.3$, `num_trials`$=5$, `num_realizations` $\in \{10, 10^2, 10^3, 10^4, 10^5, 10^6\}$. Create a figure with a 2 x 3 grid of axes objects. Each axes object should correspond to a given number of realizations.

Initialize `fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))`.

On each axes object:
- Plot a histogram of the outcomes. Use bins which are centered around integers and `density=True`. Hint: think about how many bins you need and what bins define. Label this as "simulation."
- Use a lineplot and plot the true values of the distribution using `sp.binom.pmf` vs. the number of successes. Set `color='black'` and `linewidth=1`. Label this as "theory."
- Label the y-axis "PMF" and the x-axis "# successes"
- Give each axes object a title corresponding to the number of realizations being plotted.
- Tip: an easy way to make a 2-dimensional numpy array into a 1-dimensional numpy array is by using [`np.ravel`](https://numpy.org/doc/stable/reference/generated/numpy.ravel.html)

In [ ]:
num_realizations_arr = 10**np.arange(1, 7)
n = 5
p = 0.3

results = []
for num_realizations in num_realizations_arr:
    results.append(my_binomial(rng, p, n, num_realizations))

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))

for result, ax in zip(results, np.ravel(axes)):
    outcomes = result[0]
    distribution = result[1]
    bins_centered = np.arange(np.min(outcomes), np.max(outcomes) + 2) - 0.5
    bins = bins_centered[:-1] + 0.5
    
    ax.hist(outcomes, bins=bins_centered, density=True, rwidth=0.9, label='simulation')
    ax.plot(bins, sp.binom.pmf(bins, n, p), color='black', linewidth=1, label='theory')
    ax.set_ylabel('PMF')
    ax.set_xlabel('# successes')
    ax.set_title('# realizations: ' + str(outcomes.shape[0]))
    ax.legend()
    
fig.tight_layout()
plt.show()

# Statistical quantities (statistic)
___

A statistic is a quantity which is calculated over a set of numbers, such as a sample or distribution. Because statistics are computed using a function which takes in a bunch of numbers and spits out a single (or a few) number(s), we can think of these quantities as *summarizing* something about the data.

## Mean

One of the first statistical quantities that comes to mind is the **mean**. The mean answers the question of what is the average value in the sample/distribution?

Given a sample $X$ of size $N$, we can calculate the mean using

$$
\langle x \rangle = \frac{1}{N} \sum_{i=1}^N x_i
$$

We can also create a function $g(x)$ which counts the multiplicity of the $x$ which are in in $X$. E.g., if $X = \{1, 1, 3, 3, 4, 7, 7, 7 \}$, then $g(1) = 2$, $g(3) = 2$, $g(4) = 1$, $g(7) = 3$. Any other values of $x$ are 0 since they do not appear in $X$: e.g., $g(10) = 0$. Instead of summing over each quantity in our sample, we can sum over the unique quantities in our sample and use this counting function to weight the values of $x$ accordingly:

$$
\langle x \rangle = \frac{1}{N} \sum_{x \in X} g(x)
$$

Moving $N$ inside the sum, we can recognize $g(x) / N$ as the *probability* that $x$ is in $X$ and define $P(x_i) = g(x_i) / N$. Then the mean can be computed knowing the distribution $P(x)$ and knowing what values of $x$ are or can be in $X$ (i.e., the support of $P(x)$).

$$
\langle x \rangle = \sum_{x \in X} x P(x)
$$

For non-discrete $x$, change the sum to an integral with the appropriate measure.

$$
\langle x \rangle = \int_{-\infty}^{\infty} x P(x) dx
$$

In general, for a distribution $A(x)$ which might not be normalized, $\int_{-\infty}^{\infty} A(x) \neq 1$,

$$
\langle x \rangle = \frac{\int_{-\infty}^{\infty} x A(x) dx}{\int_{-\infty}^{\infty} A(x) dx}
$$

### Write two functions to calculate the mean: one which sums over the values in the sample and one which uses the probabilities of the quantities.

In [ ]:
def sample_mean(x):
    """
    Use np.sum: https://numpy.org/doc/stable/reference/generated/numpy.sum.html
    To get the number of values in a numpy array, use x.shape[0]
    Use these two ideas to write your function.
    No need for a for loop.
    """
    pass

def distribution_mean(x, prob):
    """
    x and prob are numpy arrays. They have a direct correspondence to one another.
    I.e., the probability of x[2] is prob[2]
    Use np.sum and the direct correspondence to write your function.
    Remember that you can perform multiplication on numpy arrays!
    E.g., if a = np.array([1, 2, 3]) and b = np.array([4, 5, 6]),
    then a * b = np.array(4, 10, 18]).
    No need for a for loop.
    """
    pass

### Calculate the mean of the binomial process using $p=0.3$, $n=5$, and `num_realizations` $\in \{10, 10^2, 10^3, 10^4, 10^5, 10^6\}$. Compare your functions to `np.mean` and the analytic solution $np$. How good is the estimate of the mean at using a small number of realizations? How does the mean change as the number of realizations increases? 

In [ ]:
for (x, distribution) in results:
    distribution_x = np.arange(np.min(x), np.max(x) + 1)
    print('num realizations:', x.shape[0])
    print('sample mean:', sample_mean(x))
    print('prob mean:', distribution_mean(distribution_x, distribution))
    print('numpy mean:', np.mean(x))
    print('analytic mean:', n * p)
    print()

From now on, use `np.mean` when computing sample means.

## Variance

The variance is another statistic. It tells us about the spread of a set of numbers in relation to the set's mean. The variance is the expected squared deviation from the mean of a set $X$. Mathematically, it is

$$
\begin{align*}
\mathrm{var}(x) &= \langle \left(x - \langle x \rangle \right)^2 \rangle \\
&= \langle x^2 - 2 x \langle x \rangle + \langle x \rangle^2 \rangle \\
&= \langle x^2 \rangle - 2 \langle x \rangle^2 + \langle x \rangle^2 \\
&= \langle x^2 \rangle - \langle x \rangle^2
\end{align*}
$$

Summing over quanities in a sample, we have

$$
\mathrm{var}(x) = \frac{1}{N} \sum_{i=1}^{N} \left( x_i - \langle x \rangle \right)^2
$$

Using probability distributions, we have

$$
\mathrm{var}(x) = \int x^2 P(x) dx - \left(\int x P(x) dx \right)^2
$$

### Write two functions to calculate the variance: one which sums over the values in the sample and one which uses the probabilities of the quantities. Each function should utilize the respective function to calculate the mean.

In [ ]:
def sample_var(x):
    """
    Use your sample_mean function to get your mean.
    Again, use x.shape[0] to get the amount of values in the numpy array.
    Use np.sum to compute the sum.
    No need for any for loops.
    """
    pass

def distribution_var(x, prob):
    """
    Use your distribution_var function to get your mean.
    Use np.sum and the direct correspondence between x and prob.
    Remember numpy arrays can be squared as you would expect them to be!
    If a = np.array([2, 5, 7]), then a**2 = np.array([4, 25, 49]).
    """
    pass

### Calculate the variance of the binomial process using $p=0.3$, $n=5$, and `num_realizations` $\in \{10, 10^2, 10^3, 10^4, 10^5, 10^6\}$. Compare your functions to `np.var` and the analytic solution $np(1-p)$. How good is the estimate of the variance using a small number of realizations? How does the variance change as the number of realizations increases?

In [ ]:
for (x, distribution) in results:
    distribution_x = np.arange(np.min(x), np.max(x) + 1)
    print('num realizations:', x.shape[0])
    print('sample var:', sample_var(x))
    print('prob var:', distribution_var(distribution_x, distribution))
    print('numpy var:', np.var(x))
    print('analytic var', n * p * (1 - p))
    print()

### Repeat the above using `num_trials`$=100$.

In [ ]:
num_realizations_arr = 10**np.arange(1, 7)
n = 100
p = 0.3

results = []
for num_realizations in num_realizations_arr:
    results.append(my_binomial(rng, p, n, num_realizations))

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))
for result, ax in zip(results, axes.ravel()):
    outcomes = result[0]
    distribution = result[1]
    bins_centered = np.arange(np.min(outcomes), np.max(outcomes) + 2) - 0.5
    bins = bins_centered[:-1] + 0.5
    
    ax.hist(outcomes, bins=bins_centered, density=True, rwidth=0.9, label='simulation')
    ax.plot(bins, sp.binom.pmf(bins, n, p), color='black', linewidth=1, label='theory')
    ax.set_ylabel('PMF')
    ax.set_xlabel('# successes')
    ax.set_title('# realizations: ' + str(outcomes.shape[0]))
    ax.legend()

fig.tight_layout()
plt.show()

In [ ]:
for (x, distribution) in results:
    distribution_x = np.arange(np.min(x), np.max(x) + 1)
    print('num realizations:', x.shape[0])
    print('sample mean:', sample_mean(x))
    print('prob mean:', distribution_mean(distribution_x, distribution))
    print('numpy mean:', np.mean(x))
    print('analytic mean:', n * p)
    print()

In [ ]:
for (x, distribution) in results:
    distribution_x = np.arange(np.min(x), np.max(x) + 1)
    print('num realizations:', x.shape[0])
    print('sample var:', sample_var(x))
    print('prob var:', distribution_var(distribution_x, distribution))
    print('numpy var:', np.var(x))
    print('analytic var', n * p * (1 - p))
    print()

# Normal/Gaussian distribution
___

The normal/Gaussian distribution arises in a multitude of ways. One of these ways is through the [central limit theorem](https://en.wikipedia.org/wiki/Central_limit_theorem). The central theorem essentially states that, under certain conditions, the distribution of independent and identically distributed random variables summed together tends toward a normal distribution. For instance, simulating binomial processes with $n=100$ above, we see this bell-curve/normal behavior. Notably, a normal distribution can only **approximate** binomial processes with large $N$.

There normal distribution is parameterized by its mean $\mu$ and variance $\sigma^2$. For normally distributed $x$, its probability density is given by

$$
    P(x | \mu, \sigma^2) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp \left( \frac{-1}{2} \left( \frac{x - \mu}{\sigma} \right)^2 \right)
$$

To fit a normal distribution to data, we only need the mean and variance of the data. In practice, the normal distribution is coded up using its mean and its standard deviation (the square root of its variance), so that's the parameterization we use below for the scipy functions.

## When does a Gaussian distribution approximate a binomial distribution well?

### Let's get a feeling for how many Bernoulli trials we need to sum together for a binomial process to be approximately normal. Let $p=0.3$ and `num_realizations`=$10000$. Try out $n \in \{2, 5, 10, 20, 50, 100\}$ 

In [ ]:
num_trials_arr = np.array([2, 5, 10, 20, 50, 100])
p = 0.3
num_realizations = 10000
binomial_rands = []
for num_trials in num_trials_arr:
    binomial_rands.append(my_binomial(rng, p, num_trials, num_realizations)[0])

### Create a figure with a 2 x 3 grid of axes objects. Each axes object should correspond to a given number of trials for a binomial process. Plot a histogram of the binomial random variables using bins centered correctly. Fit a normal distribution using the mean and standard deviation of the binomial random variables and plot it.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))
for ax, rands, num_trials in zip(np.ravel(axes), binomial_rands, num_trials_arr):
    bins_centered = np.arange(np.min(rands), np.max(rands) + 2) - 0.5
    bins = bins_centered[:-1] + 0.5
    ax.hist(rands, bins=bins_centered, density=True, rwidth=0.9, label='binomial')
    ax.plot(bins, sp.norm.pdf(bins, np.mean(rands), np.std(rands)), linewidth=2, label='Gaussian fit')
    ax.set_title('$n$ = ' + str(num_trials))
    ax.set_ylabel('PMF')
    ax.set_xlabel('# successes')
    ax.legend()
    
fig.tight_layout()
plt.show()

### Repeat the same plots as above but instead use a logarithmic scale on the y-axis. Inspecting the tails of distributions is incredibly important for letting you know if your fit is good or bad.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))
for ax, rands, num_trials in zip(np.ravel(axes), binomial_rands, num_trials_arr):
    bins_centered = np.arange(np.min(rands), np.max(rands) + 2) - 0.5
    bins = bins_centered[:-1] + 0.5
    ax.hist(rands, bins=bins_centered, density=True, rwidth=0.9, label='binomial')
    ax.plot(bins, sp.norm.pdf(bins, np.mean(rands), np.std(rands)), linewidth=2, label='Gaussian fit')
    ax.set_title('$n$ = ' + str(num_trials))
    ax.set_yscale('log')
    ax.set_ylabel('PMF')
    ax.set_xlabel('# successes')
    ax.legend()
    
fig.tight_layout()
plt.show()

## When does a Gaussian approximate sums of uniform random numbers well?

The central limit theorem doesn't just apply to the binomial distribution. It applies to any idependent and identically distributed random numbers from any distribution! `rng.random()` produces random numbers uniformly distributed $\in [0, 1]$. So let's see whether they become Gaussian when we sum them together.

### Write a function to sum up $n$ uniformly distributed random numbers.

It should have three parameters:
- `rng`, a random number generator
- `n`, the amount of numbers to sum up
- `num_realizations`, the amount of times you repeat this process

It should return:
- `outcome`, a 1-dimensional numpy array containing the results for summing $n$ random numbers

In [ ]:
def my_summed_uni(rng, n, num_realizations):
    rands = rng.random(size=(num_realizations, n))
    
    """
    Use np.sum and sum the matrix of random numbers over the columns.
    Save this result to a variable called outcomes.
    """
    

    
    return outcomes

### See how many uniform random numbers you need to sum for things to appear Gaussian.

In [ ]:
ns = np.array([1, 2, 5, 10, 20, 50])
num_realizations = 10000

summed_uni_rands = []
for n in ns:
    summed_uni_rands.append(my_summed_uni(rng, n, num_realizations))
    
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))
for ax, rands, n in zip(np.ravel(axes), summed_uni_rands, ns):
    bins = np.linspace(np.min(rands), np.max(rands), 30)
    ax.hist(rands, bins=bins, density=True, rwidth=0.9, label='uni rands')
    ax.plot(bins, sp.norm.pdf(bins, np.mean(rands), np.std(rands)), linewidth=2, label='Gaussian fit')
    ax.set_title('summed ' + str(n) + ' rands')
    ax.set_xlabel('$x$')
    ax.set_ylabel('PMF')
    ax.legend()
    
fig.tight_layout()
plt.show()

### Repeat the same plots as above but instead use a logarithmic scale on the y-axis to inspect the tails.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))
for ax, rands, n in zip(np.ravel(axes), summed_uni_rands, ns):
    bins = np.linspace(np.min(rands), np.max(rands), 30)
    ax.hist(rands, bins=bins, density=True, rwidth=0.9, label='uni rands')
    ax.plot(bins, sp.norm.pdf(bins, np.mean(rands), np.std(rands)), linewidth=2, label='Gaussian fit')
    ax.set_title('summed ' + str(n) + ' rands')
    ax.set_xlabel('$x$')
    ax.set_ylabel('PMF')
    ax.set_yscale('log')
    ax.legend()
    
fig.tight_layout()
plt.show()